Extract Transforms

In [2]:
import numpy as np
import open3d as o3d
import json

# Read .ply file
transforms_path = "/home/vishalchandra/Desktop/nerfstudio_dev/outputs/exp_sapien_blade/semantic-splatfacto/config_0/dataparser_transforms.json"
splatcloud_path = "/home/vishalchandra/Desktop/nerfstudio_dev/exports/splat/splat.ply"

splatcloud = o3d.io.read_point_cloud(splatcloud_path)
# Get and invert transforms
transforms = json.load(open(transforms_path, 'r'))
inv_scale = 1.0 / transforms['scale']

linear = np.array(transforms['transform'])[0:3, 0:3]
T = np.array(transforms['transform'])[0:3, 3]
inv = np.linalg.inv(linear)

inv_hom = np.eye(4)
inv_hom[0:3, 0:3] = inv
inv_hom[0:3, 3] = -inv @ T

# apply transforms
# splatcloud.transform(inv_hom)
# splatcloud.scale(inv_scale, center=(0, 0, 0))

# correct for convention difference
# R = o3d.geometry.get_rotation_matrix_from_xyz((0, 0, np.pi/2))
# splatcloud.rotate(R, center=(0, 0, 0))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Get canonical parts

In [3]:
# Read .ply file
paris_to_sapien = {
    'blade': '103706',
    'laptop': '10211',
    'foldchair': '102255',
    'oven': '101917',
    'fridge': '10905',
    'scissor': '11100',
    'stapler': '103111',
    'USB': '100109',
    'washer': '103776',
    'storage': '45135'
}

name = 'blade'
sapien_dir = '/home/vishalchandra/Desktop/sapien_dataset/' + paris_to_sapien[name] + '/'
description = json.load(open(sapien_dir + 'result.json'))[0]
parts = description['children']

parts_files = [part['objs'] for part in parts]
parts_files[0][:5] #check

['original-3', 'original-2', 'original-5', 'original-31', 'original-32']

In [4]:
canonical_part_objs = [
    [o3d.io.read_triangle_mesh(sapien_dir + 'textured_objs/' + obj + '.obj') for obj in part]
    for part in parts_files
]
canonical_part_objs[0][:5] #check

[TriangleMesh with 290 points and 248 triangles.,
 TriangleMesh with 38 points and 30 triangles.,
 TriangleMesh with 80 points and 80 triangles.,
 TriangleMesh with 96 points and 96 triangles.,
 TriangleMesh with 72 points and 70 triangles.]

In [5]:
# add all triangle meshes to a single mesh but keep them separate
canonical_parts = []
for part in canonical_part_objs:
    mesh = o3d.geometry.TriangleMesh()
    for obj in part:
        mesh += obj
    canonical_parts.append(mesh)

# correct for convention difference between sapien and reconstruction
R = o3d.geometry.get_rotation_matrix_from_xyz((np.pi/2, -np.pi/2, 0))
for part in canonical_parts:
    part.rotate(R, center=(0, 0, 0))

Extract Reconstructed Parts from Gauss Params

In [6]:
# !!! Don't need to interact with pipelines
# from splatart.managers.SplatManager import SplatManager, load_model
# to be cmd line args
# model_dirs = [
#     '/home/vishalchandra/Desktop/nerfstudio_dev/outputs/exp_sapien_blade/semantic-splatfacto/config_0',
#     '/home/vishalchandra/Desktop/nerfstudio_dev/outputs/exp_sapien_blade/semantic-splatfacto/config_1'
# ]
# num_classes = 4
# ns_base_path = '/home/vishalchandra/Desktop/nerfstudio_dev/'

# trainer_config, pipeline, ckpt_path = load_model(model_dirs[0], 'config.yml', ns_base_path)

In [7]:
import torch

manager_paths = [
    '/home/vishalchandra/Desktop/splatart/results/sapien_exp/blade/splat_manager_0.pth',
    '/home/vishalchandra/Desktop/splatart/results/sapien_exp/blade/splat_manager_1.pth'
]

managers = [torch.load(manager_path) for manager_path in manager_paths]

In [19]:
from plyfile import PlyElement, PlyData
import numpy as np

# adapted from https://github.com/nerfstudio-project/gsplat/issues/234#issuecomment-2197277211
def part_to_ply(part, part_num):

    xyz = part["means"].detach().cpu().numpy()
    normals = np.zeros_like(xyz)
    f_dc = part["features_dc"].detach().contiguous().cpu().numpy()
    f_rest = part["features_rest"].transpose(1, 2).flatten(start_dim=1).detach().contiguous().cpu().numpy()
    opacities = part["opacities"].detach().cpu().numpy()
    scale = part["scales"].detach().cpu().numpy()
    rotation = part["quats"].detach().cpu().numpy()


    l = ['x', 'y', 'z', 'nx', 'ny', 'nz']
    # All channels except the 3 DC
    for i in range(part.features_dc.shape[1]):
        l.append('f_dc_{}'.format(i))
    for i in range(part.features_rest.shape[1]*part.features_rest.shape[2]):
        l.append('f_rest_{}'.format(i))
    l.append('opacity')
    for i in range(part.scales.shape[1]):
        l.append('scale_{}'.format(i))
    for i in range(part.quats.shape[1]):
        l.append('rot_{}'.format(i))

    dtype_full = [(attribute, 'f4') for attribute in l]

    elements = np.empty(xyz.shape[0], dtype=dtype_full)
    attributes = np.concatenate((xyz, normals, f_dc, f_rest, opacities, scale, rotation), axis=1)
    elements[:] = list(map(tuple, attributes))
    el = PlyElement.describe(elements, 'vertex')
    PlyData([el]).write('part{}.ply'.format(part_num))


for i in range(2, managers[0].num_parts):
    part = managers[0].parts_gauss_params[i]
    part_to_ply(part, i-2)

Compare recon and canonical

In [20]:
canonical_corresp = {
    0: 1,
    1: 0
}

recon_part_clouds = [
    o3d.io.read_point_cloud('part{}.ply'.format(i)) for i in range(2)
]

# apply transforms
for part_cloud in recon_part_clouds:
    part_cloud.transform(inv_hom)
    part_cloud.scale(inv_scale, center=(0, 0, 0))

canonical_part_clouds = [
    canonical_parts[canonical_corresp[i]].sample_points_uniformly(len(recon_part_clouds[i].points))
    for i in range(2)
]

In [21]:
# register moving parts
# here, we know it's part 1

print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    recon_part_clouds[1], canonical_part_clouds[1], 0.02, np.eye(4),
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
)
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=2.849162e-01, inlier_rmse=1.332728e-02, and correspondence_set size of 153
Access transformation to get result.
Transformation is:
[[ 0.99653349 -0.03472572  0.07559843 -0.01107832]
 [ 0.07064248  0.83316425 -0.5484952   0.04370705]
 [-0.04393902  0.55193429  0.83272919 -0.02639509]
 [ 0.          0.          0.          1.        ]]


In [25]:
from pytorch3d.loss import chamfer_distance

part_chamfers = []
movable = 1
for i in range(2):

    recon_cloud = recon_part_clouds[i]
    canonical_cloud = canonical_part_clouds[i]

    if i == movable:
        reg_p2p = o3d.pipelines.registration.registration_icp(
            recon_part_clouds[1], canonical_part_clouds[1], 0.02, np.eye(4),
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
        )
        recon_cloud.transform(reg_p2p.transformation)
        
    recon_T = torch.tensor(recon_cloud.points).float().unsqueeze(0)
    canonical_T = torch.tensor(canonical_cloud.points).float().unsqueeze(0)

    a, _ = chamfer_distance(recon_T, canonical_T)
    b, _ = chamfer_distance(canonical_T, recon_T)
    part_chamfers.append((a + b)/2)

part_chamfers

[tensor(0.0007), tensor(0.0349)]

In [29]:
splatcloud_path = "/home/vishalchandra/Desktop/nerfstudio_dev/exports/splat/splat.ply"
gtmesh_path = "/home/vishalchandra/Desktop/splatart_data/narf_sapien_data/v5/blade/0/blade_0.ply"

splatcloud = o3d.io.read_point_cloud(splatcloud_path)
splatcloud.transform(inv_hom)
splatcloud.scale(inv_scale, center=(0, 0, 0))
R = o3d.geometry.get_rotation_matrix_from_xyz((0, 0, np.pi/2))
splatcloud.rotate(R, center=(0, 0, 0))

gtmesh = o3d.io.read_triangle_mesh(gtmesh_path)
gtcloud = gtmesh.sample_points_uniformly(number_of_points=len(splatcloud.points))

splat_T = torch.tensor(np.array(splatcloud.points)).float().unsqueeze(0)
gt_T = torch.tensor(np.array(gtcloud.points)).float().unsqueeze(0)

a, _ = chamfer_distance(splat_T, gt_T)
b, _ = chamfer_distance(gt_T, splat_T)
chamfer_whole = (a + b) / 2
chamfer_whole

tensor(0.0007)